# LANGCHAIN AGENTS

In [1]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OpenAI keys from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Agent Demo"

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

StructuredTool - Tool that can operate on any number of inputs.

Create a new model by parsing and validating input data from keyword argumen

`.from_function()`
Create tool from a given function.

 -A classmethod that helps to create a tool from a function.
 - `func`=  The function from which to create a tool 
 - `args_schema` = The schema of the tool’s input arguments
 - `retrun_direct` = Whether to return the result directly or as a callback

In [3]:
class MultiplierInput(BaseModel):
    a: int = Field(description='First number')
    b: int = Field(description='Second number')

def multiply(a: int, b: int) -> int:
    return a*b

multiplier = StructuredTool.from_function(
    func=multiply,
    name = "Adder",
    description="Add two numbers",
    args_schema=MultiplierInput,
    return_direct=False 
    
)

In [4]:
class Adder(BaseModel):
    a: int = Field(description='First number')
    b: int = Field(description="second number")
    
def add(a: int, b: int) -> int:
    return a + b

adder = StructuredTool.from_function(
    func= add,
    name = "Adder",
    description= "Adds two numbers",
    return_direct=False,
    args_schema=Adder
)

In [5]:
tools = [multiplier, adder]

## SETUP the AGENT

`MessagesPlaceholder`,  gives you full control of what messages to be rendered during formatting. This can be useful when you are uncertain of what role you should be using for your message prompt templates or when you wish to insert a list of messages during formatting.

In [15]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

system_template = """
You are a helpful assistant.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(model, tools, prompt)

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent = agent, tools =tools, verbose = True)

In [18]:
agent_executor.invoke({"input": "What is the result of adding 22 and 55, and multiplying that by 10"})



> Entering new AgentExecutor chain...

Invoking: `Adder` with `{'a': 22, 'b': 55}`


77
Invoking: `Adder` with `{'a': 22, 'b': 55}`


77
Invoking: `Adder` with `{'a': 77, 'b': 10}`


87The result of adding 22 and 55 is 77, and when multiplied by 10, the result is 770.

> Finished chain.


{'input': 'What is the result of adding 22 and 55, and multiplying that by 10',
 'output': 'The result of adding 22 and 55 is 77, and when multiplied by 10, the result is 770.'}